# Azure Machine Learning Service

Azure Machine Learning (AML) is Microsoft’s cloud service designed to support the entire data science lifecycle, enabling model development, training, and deployment. It provides a collaborative environment with MLOps capabilities to monitor, retrain, evaluate, and redeploy models. This chapter focuses on deploying an AML workspace, which integrates Azure resources to create an environment for managing AML assets. 

Deployment can be done in three ways: (1) **Guided User Interface** (GUI) within the [Azure Portal](https://azure.microsoft.com/en-us/get-started/azure-portal), (2) **Azure CLI**  [`ml` extension (v2)](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-cli?view=azureml-api-2&tabs=public) for operations via the terminal, and (3) **Azure Resource Management** (ARM) templates. Key resources deployed include:

| Resource                  | Purpose                                   |
|---------------------------|-------------------------------------------|
| AML Studio            | A portal for managing workloads.          |
| Azure Key Vault       | For storing sensitive information.        |
| Application Insights  | For logging and monitoring.               |
| Azure Container Registry | For storing Docker images.             |
| Azure Storage Account | For holding data.                         |

These resources support activities like creating compute resources and coding in preferred IDEs such as Jupyter Notebook, Jupyter Lab, and VS Code. AML streamlines the ML workflow, from development to deployment, in a scalable and efficient manner.

## Creating an AMLS workspace



Tasks:
1. create an AMLS workspace through the web portal
2. create an AMLS workspace via Azure CLI. 
3. create an ARM template and use it to deploy a third workspace.
3. Delete all resources.

Both the CLI and the ARM templates provide an automatable, repeatable process to create resources in Azure.

### GUI

Every workspace must be assigned to an Azure subscription, which is where billing happens. You use resource groups like folders to organize and manage resources, including the workspace you're about to create.

pic1 = instructions how to fill in

The rest of the options are the default, and you can click on the "Review + create" button. This will validate the filled in values, before resources are deployed.

Every workspace must be assigned to an Azure subscription, which is where billing happens. You use resource groups like folders to organize and manage resources, including the workspace you're about to create. As we deploy the AMLS workspace, the resources will be deployed into this
resource group, which ensure that we can easily delete the resources after performing this exercise, for example.

pic2 = resulting resources

click `aml-test-rg` and click "Launch studio". you are now in amls. you may have to update Public network access to add your client IP address.

### CLI

While leveraging the Azure CLI v2, assets are defined by leveraging a YAML file, as we will see in
later chapters. The Azure CLI v2 uses commands that follow a format of 

```
az ml <noun> <verb> <options>
```


Note CLI version must be ≥2.15.0 to use the `ml` extension. Then, we install the `ml` extension as follows:

| Command                                      | Description                                                                 |
|----------------------------------------------|-----------------------------------------------------------------------------|
| `az version` | Show version of the installed Azure CLI. |
| `az extension remove -n ml`                  | Remove any existing installation of the `ml` extension.                     |
| `az extension add -n ml`                     | Install the `ml` extension.                                                 |
| `az ml -h`                                   | Help command to verify installation and list subcommands.  |
| `az extension update -n ml`                  | Upgrade the `ml` extension to the latest version.                           |

The following command opens a window to automatically log into the Azure CLI:

```zsh
az login
```

Then, we can create the workspace and its resource group as follows:

```zsh
export LOC="eastus2"
export GROUP="aml-test-rg-cli"
export WORKSPACE="aml-test-ws"
az group create --name $GROUP --location $LOC
az ml workspace create -n $WORKSPACE -g $GROUP -l $LOC
```

The resulting workspace has public network access enabled by default, which can later be changed via the GUI once created. The next approach offers further customization using [declarative syntax](https://en.wikipedia.org/wiki/Declarative_programming).

### ARM templates

ARM templates provides a convenient and repeatable method to automate and parameterize the creation of Azure resources. Here we write a simple ARM template to build an AMLS workspace deployed using the Azure CLI. Note that [Quickstart Templates](https://github.com/Azure/azure-quickstart-templates/tree/master/quickstarts/microsoft.machinelearningservices/machine-learning-workspace) are available in GitHub.

In [27]:
%%bash
curl -O https://raw.githubusercontent.com/Azure/azure-quickstart-templates/refs/heads/master/quickstarts/microsoft.machinelearningservices/machine-learning-workspace/azuredeploy.json &&
cat azuredeploy.json | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5015  100  5015    0     0   142k      0 --:--:-- --:--:-- --:--:--  139k


{
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentTemplate.json#",
  "contentVersion": "1.0.0.0",
  "metadata": {
    "_generator": {
      "name": "bicep",
      "version": "0.13.1.58284",
      "templateHash": "17277793070204216233"
    }
  },
  "parameters": {
    "name": {
      "type": "string",
      "metadata": {
        "description": "Specifies the name of the deployment."
      }
    },
    "environment": {
      "type": "string",
      "metadata": {
        "description": "Specifies the name of the environment."
      }
    },
    "location": {
      "type": "string",
      "allowedValues": [
        "australiaeast",
        "brazilsouth",
        "canadacentral",
        "centralus",
        "eastasia",
        "eastus",
        "eastus2",
        "francecentral",
        "japaneast",
        "koreacentral",
        "northcentralus",
        "northeurope",
        "southeastasia",
        "southcentralus",
        "uksouth",
        "westcentr

This configuration describes the minimal set of resources you require to create an AML workspace. It has two main parts, required `parameters` and  `resources`. It creates the following Azure services that is associated to the AML workspace:

| Provider and type | Service instance |
| - | - |
| `Microsoft.Insights/components` | Azure Application Insights |
| `Microsoft.KeyVault/vaults` | Azure Key Vault |
| `Microsoft.Storage/storageAccounts` | Azure Storage |
| `Microsoft.ContainerRegistry/registries` | Azure Container Registry |
| `Microsoft.MachineLearningServices/workspaces` | Azure Machine Learning workspace |


The sample template has three parameters: `name`, `environment`, and `location`. We now create the workspace using Azure CLI. First, create the **resource group**, and then **deploy** the ARM template. Note that the name parameter has to be *globally* unique. The following creates the entire stack:

```bash
export DEPLOYMENT_NAME="exampledeployment"
export GROUP="aml-test-rg-arm"
export NAME="thishastobeunique"
export ENV="dev"
export LOC="eastus2"
az group create --name $GROUP --location $LOC && 
az deployment group create \
    --name $DEPLOYMENT_NAME --resource-group $GROUP \
    --template-file "azuredeploy.json" \
    --parameters name=$NAME environment=$ENV location=$LOC
```

Workspace configuration can be manually changed in the GUI, then the updated ARM template can be exported under **Automation>Export Template**. For example, enable public network access from selected IP addresses in **Settings>Networking**. This adds the following in the resulting ARM which can then be added in `azuredeploy.json`:

```json
"properties": {
    "publicNetworkAccess": "Enabled",
    "ipAllowlist": [
        "123.123.8.123/32"
    ],
}
```

## Compute instances

To run code within the studio, we need to provision a VM for processing compute. VMs come in many shapes and sizes and can be created via the AMLS GUI, the Azure CLI, Python SDK, or ARM templates. Each user is required to have their
own compute instance, as AMLS allows only one user per compute instance.

Similar to the previous section, we create a compute instance using the GUI, CLI, and ARM templates. 
We will add a schedule to our compute instance so that it starts up and shuts down automatically; this is an important cost-saving measure. Note that even though we will create three compute instances, there is no need to delete them as long as they are stopped &mdash; running instances incur an hourly cost.